In [1]:
from core.project import Project
# from utils.orm import boreholes_from_dataframe
from definitions import ROOT_DIR
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_POL_LEXICON, DEFAULT_LITHO_LEGEND
import pandas as pd

In [2]:
data_dict = {'lithologies_data': f'{ROOT_DIR}/CF_data/Result_traitem/fusion_finale/Lithologies.csv',
             'pollutants_data': f'{ROOT_DIR}/CF_data/Result_traitem/fusion_finale/Samples.csv'}

In [3]:
import os
from core.orm import Base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import re
from utils.config import WARNING_TEXT_CONFIG, DEFAULT_BOREHOLE_DIAMETER, WORDS_WITH_S
from utils.utils import striplog_from_dataframe
from utils.visual import get_components, legend_from_attributes
from core.orm import BoreholeOrm, PositionOrm, LinkIntervalComponentOrm
from striplog import Component, Interval

In [4]:
def boreholes_from_dataframe(data_dict, symbols=None, attributes=None, id_col='ID',
                             diameter_col='Diameter', average_z=None, date_col='Date',
                             sample_type_col=None, verbose=False):
    """ Creates a list of BoreholeORM objects from a dataframe

    Parameters
    ----------
    data_dict: dict
        A dictionary of pandas.DataFrame containing borehole intervals data, based on the type of
        these intervals (lithology or samples). e.g: {'lithology': df_1, 'sample': df2}
    symbols: dict
        A dict e.g. {attribute_1: {'legend': striplog.Legend, 'lexicon': striplog.Lexicon}, ...}
    attributes : list
        List of dataframe's columns of interest, linked to attributes to represent like 'lithology'
    verbose : Bool
        allow verbose option if set = True

    Returns
    -------
    boreholes: list
        boreholes object
    components: dict
        dictionary containing ID and component

    """

    int_id = 0  # interval id
    pos_id = 0  # position id
    boreholes_orm = []
    components_dict = []
    comp_id = 0  # component id
    component_dict = {}
    link_intv_comp_dict = {}  # link between intervals and components (<-> junction table)
    contam_names = list(DEFAULT_POL_LEXICON.abbreviations.values())

    if len(data_dict.keys()) > 2:
        raise(KeyError("The data dictionary keys cannot be more than 2 keys"))
    if len(list(filter(re.compile('litho|sample|poll', re.I).match, data_dict.keys()))) < 1:
        raise(KeyError("data_dict keys must contain at least 'lithology', 'sample' or 'pollutant' as keywords. e.g: {'lithology_data': df_litho, 'sample_data': df_samples}"))

    # data concatenation
    df_list = []
    for k, v in data_dict.items():
        assert isinstance(v, pd.DataFrame)

        df = v.copy()
        if re.search('litho', k, re.I):
            df['_intv'] = 'lithology'
        elif re.search('sample|poll', k, re.I):
            df['_intv'] = 'sample'

        # columns' name standardization
        for col in df.columns:
            if re.search('top|toit', col, re.I):
                df.rename(columns={col: 'Top_intv'}, inplace=True)
            elif re.search('base|mur|assise', col, re.I):
                df.rename(columns={col: 'Base_intv'}, inplace=True)
            elif re.search('thick|epais', col, re.I):
                df.rename(columns={col: 'Thick_intv'}, inplace=True)
            elif re.search('desc', col, re.I):
                df.rename(columns={col: 'Desc_intv'}, inplace=True)

        df.insert(0, 'Type_intv', df.pop('_intv'))  #NOTE: Modified where the insertion is made 
        df_list.append(df)

    final_df = df_list[0].append(df_list[1])

    # data exploitation
    print(f'\nData Processing...\n================================')
    bh_id_list = []  #
    bh_counter = 0
    bh_idx = 0  # borehole index in the current dataframe

    if diameter_col not in final_df.columns:
        print(f"{WARNING_TEXT_CONFIG['blue']}"
              f"Warning : -- No borehole diameter column found or check given column's name.\n"
              f'To continue, default diameter column has been created with value: '
              f'{DEFAULT_BOREHOLE_DIAMETER} [m]{WARNING_TEXT_CONFIG["off"]}')
        final_df[diameter_col] = pd.Series([DEFAULT_BOREHOLE_DIAMETER] * len(final_df))

    top_col, base_col, desc_col = 'Top_intv', 'Base_intv', 'Desc_intv'
    thick_col, intv_type_col = 'Thick_intv', 'Type_intv'

    for idx, row in final_df.iterrows():
        bh_name = row[id_col]
        if date_col not in final_df.columns:
            bh_date = None
        else:
            bh_date = row[date_col]

        if bh_name not in bh_id_list:
            bh_id_list.append(bh_name)
            bh_selection = final_df[id_col] == f"{bh_name}"
            tmp = final_df[bh_selection].copy()
            tmp.reset_index(drop=True, inplace=True)
            striplog_dict = striplog_from_dataframe(df=tmp, bh_name=bh_name,
                                                    attributes=attributes, symbols=symbols,
                                                    id_col=id_col, thick_col=thick_col,
                                                    top_col=top_col, base_col=base_col,
                                                    desc_col=desc_col, intv_type_col=intv_type_col,
                                                    query=False, verbose=verbose)

            if striplog_dict is not None:
                bh_counter += 1
                interval_number = 0
                boreholes_orm.append(BoreholeOrm(id=bh_name, date=bh_date))

                for strip_dict in striplog_dict.values():
                    intv_type_dict = {}
                    intv_dict = {}  # just for testing
                    for iv_type, strip in strip_dict.items():
                        for c in get_components(strip):
                            c_key = list(c.keys())[0]
                            c_type = 'pollutant' if c_key in contam_names else 'lithology'
                            c_val = c_key if c_type == 'pollutant' else c[c_key]
                            # c_lev = c[c_key] if c_type == 'pollutant' else None

                            # remove 's' for plural words
                            if c_val not in WORDS_WITH_S:
                                c_val = c_val.rstrip('s')
                            c = Component({'type': c_type, 'value': c_val})
                            # c = Component({c_key: c_val}) #old
                            if c not in component_dict.keys():
                                component_dict.update({c: comp_id})
                                comp_id += 1
                                # comp_id = list(component_dict.keys()).index(c)

                        # ORM processing
                        # TODO : why error occurs when put interval_dict above, before the loop
                        interval_dict = {}
                        use_def_z = False
                        for intv in strip:
                            if average_z is not None and (row['Z'] is None or pd.isnull(row['Z'])):
                                if isinstance(average_z, int) or isinstance(average_z, float):
                                    z_val = average_z  # average Z coordinate of boreholes heads
                                    if not use_def_z:
                                        print(f"{WARNING_TEXT_CONFIG['blue']}"
                                              f"WARNING: Borehole's Z coordinate not found, use"
                                              f" default one: {average_z} [m]"
                                              f"{WARNING_TEXT_CONFIG['off']}")
                                        use_def_z = True
                                else:
                                    raise(TypeError("default_Z value must be int or float"))
                            else:
                                z_val = row['Z']

                            top = PositionOrm(id=pos_id, upper=z_val - intv.top.upper,
                                              middle=z_val - intv.top.middle,
                                              lower=z_val - intv.top.lower,
                                              x=row['X'], y=row['Y']
                                              )

                            base = PositionOrm(id=pos_id + 1, upper=z_val - intv.base.upper,
                                               middle=z_val - intv.base.middle,
                                               lower=z_val - intv.base.lower,
                                               x=row['X'], y=row['Y']
                                               )

                            desc = '; '.join([c.json() for c in intv.components])

                            interval_dict.update({int_id: {'interval_number': interval_number,
                                                    'top': top, 'base': base,
                                                    'type': iv_type, 'description': desc}})

                            intv_dict.update({int_id: {'interval_number': interval_number,
                                                           'top': top, 'base': base,
                                                           'type': iv_type, 'description': desc}})

                            update_dict(intv_type_dict, {iv_type: interval_dict})

                            for cp in intv.components:
                                if cp != Component({}):
                                    cp_key = list(cp.keys())[0]
                                    cp_type = 'pollutant' if cp_key in contam_names else 'lithology'
                                    cp_val = cp_key if cp_type == 'pollutant' else cp[cp_key]
                                    cp_lev = cp[cp_key] if cp_type == 'pollutant' else None
                                    unit = cp['unit'] if hasattr(cp, 'unit') else None
                                    pol_conc = cp['concentration'] if hasattr(cp, 'concentration') else None
                                    # remove 's' for plural words
                                    if cp_val not in WORDS_WITH_S:
                                        cp_val = cp_val.rstrip('s')
                                    cp = Component({'type': cp_type, 'value': cp_val})

                                    link_intv_comp_dict.update({(int_id, component_dict[cp]):
                                        {'extra_data': str({'level': cp_lev,
                                                            'concentration': pol_conc,
                                                            'unit': unit})}
                                                                })

                            interval_number += 1
                            int_id += 1
                            pos_id += 2

                        if bh_idx < len(boreholes_orm):
                            # TODO : find a way to store differents type of intervals in ORM
                            boreholes_orm[bh_idx].intervals_values = interval_dict
                            # boreholes_orm[bh_idx].intervals_values = intv_dict  # just for testing
                            if re.search('litho', iv_type, re.I):
                                boreholes_orm[bh_idx].litho_intv_values = intv_type_dict['lithology']
                            elif re.search('samp', iv_type, re.I):
                                boreholes_orm[bh_idx].sample_intv_values = intv_type_dict['sample']
                            else:
                                raise(TypeError(f'Unknown interval type: {iv_type}'))

                            if thick_col in final_df.columns:
                                boreholes_orm[bh_idx].length = tmp[thick_col].cumsum().max()
                            elif base_col in final_df.columns:
                                boreholes_orm[bh_idx].length = tmp[base_col].max()

                            diam_val = tmp[diameter_col][0]
                            if diam_val is not None and not pd.isnull(diam_val):
                                boreholes_orm[bh_idx].diameter = diam_val
                            else:
                                boreholes_orm[bh_idx].diameter = DEFAULT_BOREHOLE_DIAMETER
                                print(f'No diameter value found, using default: '
                                      f'{DEFAULT_BOREHOLE_DIAMETER}')

                    bh_idx += 1

            components_dict = {v: k for k, v in component_dict.items()}

    print(f"\nEnd of the process : {bh_counter} boreholes created successfully")

    return boreholes_orm, components_dict, link_intv_comp_dict

In [5]:
def create_project(data_dict, db_name, **kwargs):
    for k,v in data_dict.items():
        data_dict[k] = pd.read_csv(v, sep=',')
    verbose = kwargs.pop('verbose', False)
    sample_type_col = kwargs.pop('sample_type_col', 'Type_ech')
    diameter_col = kwargs.pop('Diam_for', 0.1) 
    pollutants = kwargs.pop('pollutants', None)
    litho_legend = kwargs.pop('litho_legend', DEFAULT_LITHO_LEGEND)
    litho_lexicon = kwargs.pop('litho_lexicon', DEFAULT_LITHO_LEXICON)
    # pollutants_legend = kwargs.pop('pollutants_legend', DEFAULT_POL_LEGEND)
    pollutants_lexicon = kwargs.pop('pollutants_lexicon', DEFAULT_POL_LEXICON)

    if pollutants is None:
        pollutants = []
        for i, c in enumerate(data_dict['pollutants_data'].columns):
            if c in pollutants_lexicon.abbreviations.keys() or c in pollutants_lexicon.abbreviations.values():
                pollutants.append(c)
    average_z = kwargs.pop('average_z', 102) 
    attributes=kwargs.pop('attributes', ['lithology']+pollutants)
    legend_dict = legend_from_attributes([('lithology', litho_legend)]+pollutants)
    symbols=kwargs.pop('symbols', {'lithology':{'lexicon': litho_lexicon}})
    boreholes, components, link_intv_comp = boreholes_from_dataframe(data_dict, verbose=verbose,
                                                sample_type_col=sample_type_col, diameter_col=diameter_col, 
                                                average_z=average_z, attributes=attributes, 
                                                symbols=symbols)

    if os.path.exists(db_name):
        os.remove(db_name)

    engine = create_engine(f"sqlite:///{db_name}", echo=True)
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    p = Project(session, name='Memoris_project', legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON)
    p.add_components(components)
    for bh in boreholes:
        p.add_borehole(bh)
    p.add_link_components_intervals(link_intv_comp)
    p.commit()
    p.refresh()
    session.close()
    p.update_legend_cmap(compute_all_attrib=True, verbose=False)
    return p

In [6]:
create_project(data_dict, 'test.db')

InvalidIndexError: Reindexing only valid with uniquely valued Index objects